In [1]:
from flask import Flask, jsonify, render_template
import csv
import pandas as pd
import numpy as np

app = Flask(__name__)

In [2]:
URL = "https://data.ca.gov/dataset/4ce08c09-1b91-4ab8-997b-20bbe88cc411/resource/eea6715b-0a13-4bfc-8e42-1cd4f8481ac8/download/dgs-telework-aggregate-by-week.csv"

data = pd.read_csv(URL)

data.rename(index = str, inplace = True, 
            columns = {"Week": "week", 
                    "Department": "dept",
                    "All Staff": "all_staff",
                    "Telework eligible staff": "tele_elig",
                    "Full-time teleworkers": "full_time_tele",
                    "Part-time teleworkers": "part_time_tele",
                    "Working days": "working_days",
                    "Telework days": "tele_days",
                    "Teleworker average one-way commute miles": "oneway_com_miles",
                    "Teleworker average one-way commute minutes": "oneway_com_minutes"
                   })

In [3]:
data

,week,dept,all_staff,tele_elig,full_time_tele,part_time_tele,working_days,tele_days,oneway_com_miles,oneway_com_minutes
0,3/16/2020,DGS,3502,1975,344,169,9750,2130,14.86,26.81
1,3/23/2020,DGS,3507,1968,1112,357,9715,6491,15.15,27.05
2,3/30/2020,DGS,3506,1968,1191,356,9715,6849,15.25,27.16
3,4/6/2020,DGS,3495,1963,1229,335,9689,6956,15.29,27.20
4,4/13/2020,DGS,3493,1962,1237,336,9686,7001,15.28,27.18
5,4/20/2020,DGS,3495,1964,1238,339,9696,7016,15.32,27.22
6,4/27/2020,DGS,3491,1959,1240,339,9671,7026,15.31,27.20
7,5/4/2020,DGS,3483,1955,1238,343,9650,7023,15.29,27.18
8,5/11/2020,DGS,3499,1970,1246,347,9723,7070,15.29,27.17
9,5/18/2020,DGS,3507,1970,1259,345,9723,7132,15.29,27.18


In [ ]:
@app.route('/')
def index():

    return render_template('home.html')

def calculate_percentage(val, total):
    """Calculates the percentage of a value over a total"""
    percent = np.divide(val, total)
    
    return percent

@app.route('/get_piechart_data')
def get_piechart_data():
    class_labels = ['Class I', 'Class II', 'Class III']
    pclass_percent = calculate_percentage(survived.groupby('Pclass').size().values, survived['PassengerId'].count())*100
    pieChartData = []
    for index, item in enumerate(pclass_percent):
        eachData = {}
        eachData['category'] = class_labels[index]
        eachData['measure'] =  round(item,1)
        pieChartData.append(eachData)

    return jsonify(pieChartData)

@app.route('/get_barchart_data')
def get_barchart_data():

    age_labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
    survived["age_group"] = pd.cut(survived.Age, range(0, 81, 10), right=False, labels=age_labels)
    survived[['age_group', 'Pclass']]

    survivorFirstClass = survived[survived['Pclass']==1]
    survivorSecondClass = survived[survived['Pclass']==2]
    survivorThirdClass = survived[survived['Pclass']==3]

    survivorAllclassPercent = calculate_percentage(survived.groupby('age_group').size().values,survived['PassengerId'].count())*100
    survivorFirstclassPercent = calculate_percentage(survivorFirstClass.groupby('age_group').size().values,survivorFirstClass['PassengerId'].count())*100
    survivorSecondclassPercent = calculate_percentage(survivorSecondClass.groupby('age_group').size().values,survivorSecondClass['PassengerId'].count())*100
    survivorThirdclassPercent = calculate_percentage(survivorThirdClass.groupby('age_group').size().values,survivorThirdClass['PassengerId'].count())*100

    barChartData = []
    for index, item in enumerate(survivorAllclassPercent):
        eachBarChart = {}
        eachBarChart['group'] = "All"
        eachBarChart['category'] = age_labels[index]
        eachBarChart['measure'] = round(item,1)
        barChartData.append(eachBarChart)


    for index, item in enumerate(survivorFirstclassPercent):
        eachBarChart = {}
        eachBarChart['group'] = "Class I"
        eachBarChart['category'] = age_labels[index]
        eachBarChart['measure'] = round(item,1)
        barChartData.append(eachBarChart)

    for index, item in enumerate(survivorSecondclassPercent):
        eachBarChart = {}
        eachBarChart['group'] = "Class II"
        eachBarChart['category'] = age_labels[index]
        eachBarChart['measure'] = round(item,1)
        barChartData.append(eachBarChart)

    for index, item in enumerate(survivorThirdclassPercent):
        eachBarChart = {}
        eachBarChart['group'] = "Class III"
        eachBarChart['category'] = age_labels[index]
        eachBarChart['measure'] = round(item,1)
        barChartData.append(eachBarChart)
    
    return jsonify(barChartData)


if __name__ == '__main__':
      app.run(debug=True)